#### 1. Jieba最大概率法分词

In [1]:
# -*- coding: utf-8 -*-
import sys
import os.path
import math
#import nltk
#from nltk.corpus import PlaintextCorpusReader
 
path1 = './icwb2-data-master/gold/pku_training_words.utf8'
path2 = './icwb2-data-master/training/pku_training.utf8'
path3 = './icwb2-data-master/testing/pku_test.utf8'
 
 
with open(path1, 'rt', encoding='utf-8') as f:
    training_words = [w.strip() for w in f.readlines()]
 
#training = PlaintextCorpusReader( *os.path.split(path2) )
#print(training)
#training_words += list(training.words())

with open(path2,'rt',encoding='utf-8') as f:
    for line in f:
        training_words += line.split('  ')
N = len(training_words)
V = len( set(training_words) )
#fdist = nltk.FreqDist(training_words)
fdist = {}
for word in training_words:
    if word not in fdist:
        fdist[word]=1
    else:
        fdist[word]+=1
fdist = dict([(w, math.log((c+1.0)/(N+V))) for w, c in fdist.items()])
defprob = math.log(1.0/(N+V))
 
with open(path3, 'rt', encoding='utf8') as f:
    test = f.readlines()
 
def get_DAG(sentence):
    DAG = {}
    T = len(sentence)
    for x in range(T):
        ys = []
        for y in range(x+1, T+1):
            if sentence[x:y] in fdist:
                ys.append(y)
        if not ys:
            ys.append(x+1)
        DAG[x] = ys
    return DAG

'''
深度优先搜索
'''
def dfs(DAG, sentence):
    segments = []
    T = len(sentence)
    def _dfs(words, x):
        for y in DAG[x]:
            if y < T:
                new = words.copy()
                new.append(sentence[x:y])
                _dfs(new, y)
            else:
                new = words.copy()
                new.append(sentence[x:y])
                segments.append( new )
    _dfs([], 0)
    bestseg = max([(sum(fdist.get(w, defprob) for w in seg), seg) for seg in segments])
    return bestseg[1]

'''
动态规划搜索最大权重
'''
def dp(DAG, sentence):
    T = len(sentence)
    prob = {T:(0.0,)}
    for x in range(T-1, -1, -1):
        prob[x] = max([(fdist.get(sentence[x:y], defprob) + prob[y][0], y) for y in DAG[x]])
    x = 0
    bestseg = []
    while x < T:
        y = prob[x][1]
        bestseg.append( sentence[x:y] )
        x = y
    return bestseg

'''
分词测试
'''
for sent in test[:5]:
    DAG = get_DAG(sent)
    print(DAG)
    #seg1 = dfs(DAG, sent)
    seg2 = dp(DAG, sent)
    #print('  '.join(seg1), sep='', end='')
    print('  '.join(seg2), sep='', end='')
    #break

{0: [1, 2], 1: [2, 3], 2: [3, 4], 3: [4], 4: [5, 6], 5: [6], 6: [7], 7: [8, 10], 8: [9, 10], 9: [10], 10: [11, 12], 11: [12], 12: [13], 13: [14], 14: [15], 15: [16], 16: [17], 17: [18, 19], 18: [19], 19: [20, 21], 20: [21], 21: [22]}
共同  创造  美好  的  新世纪  ——  二  ○  ○  一  年  新年  贺词  
{0: [1], 1: [2, 6], 2: [3], 3: [4], 4: [5], 5: [6], 6: [7, 8, 9], 7: [8, 9], 8: [9], 9: [10, 11, 12, 13], 10: [11, 12, 13], 11: [12, 13], 12: [13], 13: [14], 14: [15], 15: [16], 16: [17, 18], 17: [18], 18: [19], 19: [20], 20: [21], 21: [22]}
（  二○○○年  十二月  三十一日  ）  （  附  图片  1  张  ）  
{0: [1, 2], 1: [2], 2: [3], 3: [4], 4: [5, 6], 5: [6], 6: [7], 7: [8], 8: [9, 10], 9: [10], 10: [11], 11: [12], 12: [14], 13: [14], 14: [15], 15: [16], 16: [17]}
女士  们  ，  先生  们  ，  同志  们  ，  朋友  们  ：  
{0: [1], 1: [2], 2: [3], 3: [4], 4: [5], 5: [6, 7], 6: [7], 7: [8, 9], 8: [9], 9: [10, 11], 10: [11], 11: [12, 13], 12: [13], 13: [14], 14: [15, 16], 15: [16], 16: [17, 18], 17: [18, 19], 18: [19, 20], 19: [20], 20: [21], 21: [23

#### 2. jieba最大概率分词方法分步解析

In [2]:
path1 = './icwb2-data-master/gold/pku_training_words.utf8'
path2 = './icwb2-data-master/training/pku_training.utf8'
path3 = './icwb2-data-master/testing/pku_test.utf8'

'''
读取词典
'''
with open(path1, 'rt', encoding='utf-8') as f:
    training_words = [w.strip() for w in f.readlines()]
print('词典中的最后10个词：\n',training_words[-10:])
print('词典中的总词数：\n',len(training_words))

词典中的最后10个词：
 ['鲁伦', '听筒', '供过于求', '牵挂', '照', '莺莺', '煤', '表皮', '煮', '宏愿']
词典中的总词数：
 55303


In [3]:
'''
将训练语料中的词加入词典
'''
with open(path2,'rt',encoding='utf-8') as f:
    for line in f:
        training_words += line.split('  ')
print('加入训练语料之后的词典中的最后10个词：\n',training_words[-10:])
print('加入训练语料之后的词典中的总词数：\n',len(training_words))

加入训练语料之后的词典中的最后10个词：
 ['，', '\n', '才', '发觉', '已', '迷失', '了', '来路', '。', '\n']
加入训练语料之后的词典中的总词数：
 1184306


In [4]:
N = len(training_words)
V = len( set(training_words) )
print('总词频：\n',N)
print('总词数：\n',V)

总词频：
 1184306
总词数：
 55325


In [5]:
import math
fdist = {}
for word in training_words:
    if word not in fdist:
        fdist[word]=1
    else:
        fdist[word]+=1
fdist = dict([(w, math.log((c+1.0)/(N+V))) for w, c in fdist.items()])
defprob = math.log(1.0/(N+V))
print(len(fdist))

55325


In [6]:
with open(path3, 'rt', encoding='utf8') as f:
    test = f.readlines()

In [7]:
def get_DAG(sentence):
    DAG = {}
    T = len(sentence)
    for x in range(T):
        ys = []
        for y in range(x+1, T+1):
        #for y in range(x, T):
            if sentence[x:y] in fdist:
                ys.append(y)
        if not ys:
            ys.append(x+1)
        DAG[x] = ys
    return DAG

In [8]:
def dfs(DAG, sentence):
    segments = []
    T = len(sentence)
    def _dfs(words, x):
        for y in DAG[x]:
            if y < T:
                new = words.copy()
                new.append(sentence[x:y])
                _dfs(new, y)
            else:
                new = words.copy()
                new.append(sentence[x:y])
                segments.append( new )
    _dfs([], 0)
    bestseg = max([(sum(fdist.get(w, defprob) for w in seg), seg) for seg in segments])
    return bestseg[1]

In [9]:
def dp(DAG, sentence):
    T = len(sentence)
    prob = {T:(0.0,)}
    for x in range(T-1, -1, -1):
        prob[x] = max([(fdist.get(sentence[x:y], defprob) + prob[y][0], y) for y in DAG[x]])
    x = 0
    bestseg = []
    while x < T:
        y = prob[x][1]
        bestseg.append( sentence[x:y] )
        x = y
    return bestseg

In [10]:
sent = "去北京大学玩"
print(sent)
DAG = get_DAG(sent)
print(DAG)

去北京大学玩
{0: [1], 1: [2, 3, 5], 2: [3], 3: [4, 5], 4: [5], 5: [6]}


In [11]:
sent = "线程是程序执行时的最小单位，它是进程的一个执行流，"

In [12]:
import time
start = time.time()
DAG = get_DAG(sent)
seg1 = dfs(DAG, sent)
end = time.time()
print(end-start)
print('  '.join(seg1), sep='', end='')


0.0009970664978027344
线  程  是  程序  执行  时  的  最  小  单位  ，  它  是  进程  的  一个  执行  流  ，

In [13]:
import time
start = time.time()
DAG = get_DAG(sent)
seg2 = dp(DAG, sent)
end = time.time()
print(end-start)
print('  '.join(seg2), sep='', end='')


0.0009980201721191406
线  程  是  程序  执行  时  的  最  小  单位  ，  它  是  进程  的  一个  执行  流  ，

#### HMM分词与序列标注

In [14]:
# -*- coding: utf-8 -*-
import sys, re, math
 
#sys.argv.append('./gold/pku_training_words.utf8')
#sys.argv.append('./training/pku_training.utf8')
#sys.argv.append('./testing/pku_test.utf8')

training_words_filename = './icwb2-data-master/gold/pku_training_words.utf8'
training_filename = './icwb2-data-master/training/pku_training.utf8'
test_filename = './icwb2-data-master/testing/pku_test.utf8'

 
with open(training_words_filename, 'rt', encoding='utf8') as f:
    training_words = f.readlines()
 
with open(training_filename, 'rt', encoding='utf8') as f:
    training = f.readlines()
 
with open(test_filename, 'rt', encoding='utf8') as f:
    test = f.readlines()
 
# word tag by char
A, B, P = {}, {}, {}
for line in training:
    #print( line )
    prev_a = None
    for w, word in enumerate(re.split(r'\s{2}', line)):
        I = len(word)
        for i, c in enumerate(word):
            if I == 1:
                a = 'S'
            else:
                if i == 0:
                    a = 'B'
                elif i == I-1:
                    a = 'E'
                else:
                    a = 'M'
            # print(w, i, c, a)
            if prev_a is None: # calculate Initial state Number
                if a not in P: 
                    P[a] = 0
                P[a] += 1
            else: # calculate State transition Number
                if prev_a not in A: 
                    A[prev_a] = {}
                if a not in A[prev_a]: 
                    A[prev_a][a] = 0
                A[prev_a][a] += 1
            prev_a = a
            # calculate Observation Number
            if a not in B: 
                B[a] = {}
            if c not in B[a]: 
                B[a][c] = 0
            B[a][c] += 1
 
# calculate probability
for k, v in A.items():
    total = sum(v.values())
    A[k] = dict([(x, math.log(y / total)) for x, y in v.items()])

for k, v in B.items():
    # plus 1 smooth
    total = sum(v.values())
    V = len(v.values())
    B[k] = dict([(x, math.log((y+1.0) / (total+V))) for x, y in v.items()])
    # plus 1 smooth
    B[k]['<UNK>'] = math.log(1.0 / (total+V))

minlog = math.log( sys.float_info.min )
total = sum(P.values())
for k, v in P.items():
    P[k] = math.log( v / total )
 
def viterbi(observation):
    #P(O|I,\lambda)
    state = ['B','M','E','S']
    T = len(observation)
    delta = [None] * (T + 1)
    psi = [None] * (T + 1)
    delta[0] = dict([(i, P.get(i, minlog) + B[i].get(observation[0], B[i]['<UNK>'])) for i in state])
    psi[0] = dict([(i, None) for i in state])
    for t in range(1, len(observation)):
        Ot = observation[t]
        delta[t] = dict([(j, max([delta[t-1][i] + A[i].get(j, minlog) + B[j].get(Ot, B[j]['<UNK>']) for i in state])) for j in state])
        psi[t] = dict([(j, max([(delta[t-1][i] + A[i].get(j, minlog), i) for i in state])[1]) for j in state ])
    delta[T] = max( [ delta[T-1][i] for i in state ] )
    psi[T] = max([(delta[T-1][i], i) for i in state ] )[1]
    q = [None] * (T+1)
    q[T] = psi[T]
    for t in range(T-1, -1, -1):
        q[t] = psi[t][q[t+1]]
    return q[1:]
 
for sent in test[:5]:
    sequence = viterbi( list(sent) )
    segment = []
    for char, tag in zip(sent, sequence):
        if tag == 'B':
            segment.append(char)
        elif tag == 'M':
            segment[-1] += char
        elif tag == 'E':
            segment[-1] += char
        elif tag == 'S':
            segment.append(char)
        else:
            raise Exception()
    print('  '.join(segment), sep='', end='')
    #break

共同  创造  美好  的  新  世纪  ——二○○一年  新年  贺词  
（  二○○○年  十二月  三十  一日  ）  （  附  图片  1张  ）  
女士  们  ，  先  生们  ，  同志  们  ，  朋友  们  ：  
2001年  新年  钟声  即  将  敲响  。  人类  社会  前进  的  航船  就  要驶入21世纪  的  新  航程  。  中国  人民  进入  了  向  现代化  建设  第三步  战略  目标  迈进  的  新  征程  。  
在  这个  激动  人心  的  时刻  ，  我  很  高兴  通过  中国  国际  广播  电台  、  中央  人民  广播  电台  和  中央  电视  台  ，  向  全国  各族  人民  ，  向  香港  特别  行政区  同胞  、  澳门  特别  行政区  同胞  和  台湾  同胞  、  海外  侨胞  ，  向  世界  各国  的  朋友  们  ，  致以  新  世纪  第一  个  新年  的  祝贺  ！  


In [15]:
from jieba import analyse
# 引入TextRank关键词抽取接口
textrank = analyse.textrank
 
# 原始文本
text = "线程是程序执行时的最小单位，它是进程的一个执行流，\
        是CPU调度和分派的基本单位，一个进程可以由很多个线程组成，\
        线程间共享进程的所有资源，每个线程有自己的堆栈和局部变量。\
        线程由CPU独立调度执行，在多CPU环境下就允许多个线程同时运行。\
        同样多线程也可以实现并发操作，每个请求分配一个线程来处理。"
 
print("\nkeywords by textrank:")
# 基于TextRank算法进行关键词抽取
keywords = textrank(text)
# 输出抽取出的关键词
for keyword in keywords:
    print(keyword + "/")

Building prefix dict from the default dictionary ...



keywords by textrank:


Dumping model to file cache C:\Users\zhang\AppData\Local\Temp\jieba.cache
Loading model cost 0.781 seconds.
Prefix dict has been built successfully.


线程/
进程/
调度/
单位/
操作/
请求/
分配/
允许/
基本/
并发/
独立/
资源/
共享/
执行/
堆栈/
分派/
运行/
实现/
处理/
程序执行/


In [29]:
!pip install jieba

distributed 1.21.8 requires msgpack, which is not installed.
twisted 20.3.0 has requirement attrs>=19.2.0, but you'll have attrs 18.1.0 which is incompatible.
automat 20.2.0 has requirement attrs>=19.2.0, but you'll have attrs 18.1.0 which is incompatible.
You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [17]:
from jieba import analyse
# 引入TF-IDF关键词抽取接口
tfidf = analyse.extract_tags
 
# 原始文本
text = "线程是程序执行时的最小单位，它是进程的一个执行流，\
        是CPU调度和分派的基本单位，一个进程可以由很多个线程组成，\
        线程间共享进程的所有资源，每个线程有自己的堆栈和局部变量。\
        线程由CPU独立调度执行，在多CPU环境下就允许多个线程同时运行。\
        同样多线程也可以实现并发操作，每个请求分配一个线程来处理。"
 
# 基于TF-IDF算法进行关键词抽取
keywords = tfidf(text)
print("keywords by tfidf:")
# 输出抽取出的关键词
for keyword in keywords:
    print(keyword + "/")

keywords by tfidf:
线程/
CPU/
进程/
调度/
多线程/
程序执行/
每个/
执行/
堆栈/
局部变量/
单位/
并发/
分派/
一个/
共享/
请求/
最小/
可以/
允许/
分配/


In [ ]:
TFIDF
TextRanK
HMM-POS

In [ ]:
PKUseg
Thu
